### Data Wrangling Steps
#### By: Irene Yao
#### Date: 8/20/2018
The data is downloaded into csv files from DonorsChoose.org: https://research.donorschoose.org/t/download-opendata/33     
For this project, I will use the two datasets: Project Data and Donation Data.     

#### Step 1: Import relevant packages

In [1]:
import pandas as pd
import numpy as np

#### Step 2: Inspect columns from both data tables

In [2]:
donation_file = 'donations.csv'
with open(donation_file) as dfile:
    columns = dfile.readline() ## read the first line, which is the columns
    print(columns)

,_donationid,_projectid,_donor_acctid,_cartid,donor_city,donor_state,donor_zip,is_teacher_acct,donation_timestamp,donation_to_project,donation_optional_support,donation_total,donation_included_optional_support,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,is_teacher_referred,giving_page_id,giving_page_type,for_honoree,thank_you_packet_mailed



In [3]:
projects_file = 'projects.csv'
with open(projects_file) as pfile:
    columns = pfile.readline() ## read the first line, which is the columns
    print(columns)

,_projectid,_teacher_acctid,_schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,vendor_shipping_charges,sales_tax,payment_processing_charges,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration



#### Step 3: Merge donation and project tables and remove the irrelevant columns
Some of the columns are created to link to other tables which I'm not going to cover in this study. Those columns will be removed.      
After inspecting the two data tables, the following columns are determined neccessary.    
* From donation table: '_donationid','_projectid','_donor_acctid','donor_city','donor_state','is_teacher_acct','donation_timestamp','donation_to_project',
'donation_optional_support','donation_total','payment_method','is_teacher_referred','thank_you_packet_mailed' 

* From projects table: '_projectid','_teacher_acctid','school_ncesid','school_latitude','school_longitude','school_city','school_state','school_zip','school_metro',
'school_district','school_county','teacher_prefix','primary_focus_subject','primary_focus_area','secondary_focus_subject','secondary_focus_area',
'resource_type','poverty_level','grade_level','total_price_excluding_optional_support','total_price_including_optional_support','students_reached',
'total_donations','num_donors','eligible_double_your_impact_match','eligible_almost_home_match','funding_status','date_posted','date_completed',
'date_thank_you_packet_mailed','date_expiration'

##### a. Import the donation dataset   
   The donation dataset consists information of each donation from qualified donors. It includes data aspects such as donor    location, teacher status, and donation amount. Since the dataset is very large, I will define the data types for each column at the time of the import.

In [5]:
## import csv for donation
chunksize = 10000
chunks = []
for chunk in pd.read_csv(donation_file, chunksize=chunksize):
    chunks.append(chunk)
df_donation = pd.concat(chunks, axis=0)

##### b. Import the project dataset  
The project dataset comprises data on the project itself. Features include project status, total donation amount, and project subject, etc. 

In [6]:
## import csv for projects
chunsize = 10000
chunks = []
for chunk in pd.read_csv(projects_file, chunksize=chunksize):
    chunks.append(chunk)
df_projects = pd.concat(chunks, axis=0)

##### c. Selecting the relevant columns

In [7]:
## select only the relevant columns
## zipcode column is removed because the last two digits are hidden; didn't offer useful info
new_cols_d = ['_donationid','_projectid','_donor_acctid','donor_city','donor_state','is_teacher_acct'
            ,'donation_timestamp','donation_to_project','donation_optional_support','donation_total','payment_method','is_teacher_referred'
            ,'thank_you_packet_mailed']
df_donation_short = df_donation[new_cols_d]
df_donation_short.head(2)

,_donationid,_projectid,_donor_acctid,donor_city,donor_state,is_teacher_acct,donation_timestamp,donation_to_project,donation_optional_support,donation_total,payment_method,is_teacher_referred,thank_you_packet_mailed
0,0000023f507999464aa2b78875b7e5d6,69bf3a609bb4673818e0eebd004ea504,22c50856b0824db76daf527da6af9abf,NaN,NaN,f,2011-02-13 11:07:19.349,8.50,1.50,10.0,creditcard,f,f
1,000009891526c0ade7180f8423792063,26f02742185eb1f73f3bc5be4655fae2,c91489d7b6b89943a28555e6add72509,NaN,NJ,t,2013-05-26 11:28:31.30,63.75,11.25,75.0,creditcard,f,f


In [8]:
## do the same for projects dataset
new_cols_p = ['_projectid','_teacher_acctid','school_ncesid','school_latitude','school_longitude'
              ,'school_city','school_state','school_zip','school_metro','school_district','school_county'
              ,'teacher_prefix','primary_focus_subject','primary_focus_area','secondary_focus_subject','secondary_focus_area'
              ,'resource_type','poverty_level','grade_level','total_price_excluding_optional_support'
              ,'total_price_including_optional_support','students_reached','total_donations','num_donors'
              ,'eligible_double_your_impact_match','eligible_almost_home_match','funding_status','date_posted'
              ,'date_completed','date_thank_you_packet_mailed','date_expiration']
df_projects_short = df_projects[new_cols_p]
df_projects_short.head(2)

,_projectid,_teacher_acctid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,...,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration
0,7342bd01a2a7725ce033a179d22e382d,5c43ef5eac0f5857c266baa1ccfa3d3f,3.600097e+11,40.688454,-73.910432,New York City,NY,11207.0,urban,New York City Dept Of Ed,...,0.0,251.9,1,f,f,completed,2002-09-13 00:00:00,2002-09-23 00:00:00,2003-01-27 00:00:00,2003-12-31 00:00:00
1,ed87d61cef7fda668ae70be7e0c6cebf,1f4493b3d3fe4a611f3f4d21a249376a,3.600077e+11,40.765517,-73.960090,New York City,NY,10065.0,NaN,New York City Dept Of Ed,...,0.0,137.0,1,f,f,completed,2002-09-13 00:00:00,2002-09-23 00:00:00,2003-01-03 00:00:00,2003-12-31 00:00:00


##### d. Merge the donation and project tables

In [9]:
## merge projects table together with donation table using _projectid
df = df_donation_short.merge(df_projects_short, how='inner', on='_projectid')
df.head()

,_donationid,_projectid,_donor_acctid,donor_city,donor_state,is_teacher_acct,donation_timestamp,donation_to_project,donation_optional_support,donation_total,...,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration
0,0000023f507999464aa2b78875b7e5d6,69bf3a609bb4673818e0eebd004ea504,22c50856b0824db76daf527da6af9abf,NaN,NaN,f,2011-02-13 11:07:19.349,8.50,1.50,10.00,...,23.0,510.90,4,t,f,completed,2011-01-23 00:00:00,2011-03-18 00:00:00,2011-05-10 00:00:00,2011-06-21 00:00:00
1,53ec9a692cd770d6e4f0c6673451ff60,69bf3a609bb4673818e0eebd004ea504,ba7d4afdfc182c4c5fde1d57980697bc,NaN,CA,f,2011-03-18 01:54:04.96,217.13,38.32,255.45,...,23.0,510.90,4,t,f,completed,2011-01-23 00:00:00,2011-03-18 00:00:00,2011-05-10 00:00:00,2011-06-21 00:00:00
2,798dad82b651ff0371e4a655e56bbca5,69bf3a609bb4673818e0eebd004ea504,9b29654e7ea1241e6fa1ec4805b7429e,Wilton,CA,f,2011-03-18 01:54:04.882,166.13,29.32,195.45,...,23.0,510.90,4,t,f,completed,2011-01-23 00:00:00,2011-03-18 00:00:00,2011-05-10 00:00:00,2011-06-21 00:00:00
3,87c43e67b49398b4e0d54d31e2ae95ca,69bf3a609bb4673818e0eebd004ea504,b8f54362e335b81171ebbe36c657ea4b,Orangevale,CA,f,2011-01-31 00:41:24.833,42.50,7.50,50.00,...,23.0,510.90,4,t,f,completed,2011-01-23 00:00:00,2011-03-18 00:00:00,2011-05-10 00:00:00,2011-06-21 00:00:00
4,000009891526c0ade7180f8423792063,26f02742185eb1f73f3bc5be4655fae2,c91489d7b6b89943a28555e6add72509,NaN,NJ,t,2013-05-26 11:28:31.30,63.75,11.25,75.00,...,78.0,335.96,4,f,f,completed,2013-02-11 00:00:00,2013-05-26 00:00:00,2013-09-09 00:00:00,2013-06-11 00:00:00


Inner join donation table and project table should return the same number of rows as donation table itself, check this by viewing the shape of the 3 tables.

In [10]:
print(df_donation_short.shape)
print(df_projects_short.shape)
print(df.shape)

(6211956, 13)
(1203287, 31)
(6211956, 43)


In [12]:
print(df['date_posted'].min())
print(df['date_posted'].max())

2002-09-13 00:00:00
2016-10-11 00:00:00


We notice that the merged dataset has more than 6 million entries for projects entered from year 2002 to 2016. For the purpose of the study, we will only look at projects that are posted after 2015 to speed up the analysis. 

## Data Wrangling
Now we can put the donations and projects tables aside for a while and work on the merged table for data wrangling. 

#### Step 1: Inspect the dataframe

In [11]:
## inspect the dataframe
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6211956 entries, 0 to 6211955
Data columns (total 43 columns):
_donationid                               object
_projectid                                object
_donor_acctid                             object
donor_city                                object
donor_state                               object
is_teacher_acct                           object
donation_timestamp                        object
donation_to_project                       float64
donation_optional_support                 float64
donation_total                            float64
payment_method                            object
is_teacher_referred                       object
thank_you_packet_mailed                   object
_teacher_acctid                           object
school_ncesid                             float64
school_latitude                           float64
school_longitude                          float64
school_city                               object

The dataset has 6211956 rows and some of the columns have null values. We notice that the biggest problem with the dataset is that it consumes more than 12.9GB of memory. Before we take care of the missing values, we need to address the memory problem first. 

#### Step 2: Optimize and downsize the dataset

The dataset has 3 data types: float, integer and object. First, let's inspect the average memory usage for each data type. 

In [13]:
for dtype in ['float','integer','object']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 47.39 MB
Average memory usage for integer columns: 29.62 MB
Average memory usage for object columns: 398.94 MB


In [12]:
## downsizing the integer and float columns to numeric
df_int = df.select_dtypes(include=['integer'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')

Since many columns in the dataset are categorical data, converting them to the type 'category' will save a lot of memory. However, we don't want to convert all the object columns to 'category' if they don't have a few unique values. As a result, we only want to convert columns where less than 50% of the values are unique. 

In [13]:
converted_obj = pd.DataFrame()
df_obj = df.select_dtypes(include=['object'])

for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

As a final step, we replace the orginal columns with the converted optimized columns. 

In [15]:
df[converted_int.columns] = converted_int
df[converted_float.columns] = converted_float
df[converted_obj.columns] = converted_obj

After the optimization, we can see that the memory usage has decreased from 12.9GB to 2GB. 

In [16]:
## inspect the dataframe
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6211956 entries, 0 to 6211955
Data columns (total 43 columns):
_donationid                               object
_projectid                                category
_donor_acctid                             category
donor_city                                category
donor_state                               category
is_teacher_acct                           category
donation_timestamp                        object
donation_to_project                       float32
donation_optional_support                 float32
donation_total                            float32
payment_method                            category
is_teacher_referred                       category
thank_you_packet_mailed                   category
_teacher_acctid                           category
school_ncesid                             float32
school_latitude                           float32
school_longitude                          float32
school_city                   

#### Step 3: Cleanup the date columns

In [17]:
df['donation_timestamp'] = pd.to_datetime(df['donation_timestamp'])
df['date_posted'] = pd.to_datetime(df['date_posted'])
df['date_completed'] = pd.to_datetime(df['date_completed'])
df['date_thank_you_packet_mailed'] = pd.to_datetime(df['date_thank_you_packet_mailed'])
df['date_expiration'] = pd.to_datetime(df['date_expiration'])

#### Step 4: Deal with missing values

a. Finding the columns that have missing values

In [18]:
nan_cols = []
for col in df.columns:
    if df[col].count()<df['_donationid'].count():
        nan_cols.append(col)
print(nan_cols)

['donor_city', 'donor_state', 'school_ncesid', 'school_city', 'school_zip', 'school_metro', 'school_district', 'school_county', 'teacher_prefix', 'primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'grade_level', 'students_reached', 'date_completed', 'date_thank_you_packet_mailed', 'date_expiration']


b. Fill in null values for school_city   
For the missing values in school_city, I would like to see if the corresponding school_district is known. If yes, we can map the city according to the school district.

In [19]:
## check if any entry has a missing school_city, but a known school_district
missing_city = df[(df['school_city'].isnull()) & (df['school_district'].notnull())]
missing_city['school_district'].unique()

[Dc Public Schools, [State Run], Texas Dept Of Education, San Francisco Unified Sch Dist, Gary Cmty School District, ..., Chicago Public School Dist 299, San Jose Unified School Dist, School City Of East Chicago, Lafayette School District, Oxford School District]
Length: 30
Categories (30, object): [Dc Public Schools, [State Run], Texas Dept Of Education, San Francisco Unified Sch Dist, ..., San Jose Unified School Dist, School City Of East Chicago, Lafayette School District, Oxford School District]

The above is a list of school districts that correspond to a missing school_city. We can create a dictionary with school districts as the keys and city as the values. Then we can use the mapping to fill in some of the NaN values in the school_city column. 

In [20]:
d = {}
d['Dc Public Schools'] = 'Washington'
d['San Francisco Unified Sch Dist'] = 'San Francisco'
d['Gary Cmty School District'] = 'Gary'
d['Philadelphia City School Dist'] = 'Philadelphia'
d['Santa Clara Unified Sch Dist'] = 'Santa Clara'
d['Sonoma Valley Unified Sch Dist'] = 'Sonoma'
d['St Louis City Public Sch Dist'] = 'St. Louis'
d['Laguna Joint Elem School Dist'] = 'Petaluma'
d['Indianapolis Public Sch Dist'] = 'Indianapolis'
d['Los Angeles Unif Sch Dist'] = 'Los Angeles'
d['Detroit Public School District'] = 'Detroit'
d['Baltimore City Public Sch Dist'] = 'Baltimore'
d['Oakland Unified School Dist'] = 'Oakland'
d['Chicago Public School Dist 299'] = 'Chicago'
d['San Jose Unified School Dist'] = 'San Jose'
d['School City Of East Chicago'] = 'East Chicago'
d['Lafayette School District'] = 'Lafayette'
d['Oxford School District'] = 'Oxford'
print(d)

{'Dc Public Schools': 'Washington', 'San Francisco Unified Sch Dist': 'San Francisco', 'Gary Cmty School District': 'Gary', 'Philadelphia City School Dist': 'Philadelphia', 'Santa Clara Unified Sch Dist': 'Santa Clara', 'Sonoma Valley Unified Sch Dist': 'Sonoma', 'St Louis City Public Sch Dist': 'St. Louis', 'Laguna Joint Elem School Dist': 'Petaluma', 'Indianapolis Public Sch Dist': 'Indianapolis', 'Los Angeles Unif Sch Dist': 'Los Angeles', 'Detroit Public School District': 'Detroit', 'Baltimore City Public Sch Dist': 'Baltimore', 'Oakland Unified School Dist': 'Oakland', 'Chicago Public School Dist 299': 'Chicago', 'San Jose Unified School Dist': 'San Jose', 'School City Of East Chicago': 'East Chicago', 'Lafayette School District': 'Lafayette', 'Oxford School District': 'Oxford'}


In [57]:
## add new cities to the school_city category if not already there
city_cat = df['school_city'].cat.categories
new_cities = []
for key, value in d.items():
    new_cities.append(value)   

for i in new_cities:
    if i not in city_cat:
        df['school_city'].cat.add_categories([i], inplace = True)

In [59]:
## replace the null values in 'school_city' column
def city(x):
    if x in d.keys():
        return d[x]
    else: 
        return

df.loc[(df['school_city'].isnull()), 'school_city'] = df[df['school_city'].isnull()]['school_district'].apply(lambda x: city(x))

Some of the school districts cover more than one city. Thus, we can't infer the corresponding city from the district name. By applying the above mapping, we were able to decrease the null value in the school_city column from more than 50,000 to 12,673. 

In [60]:
df[df['school_city'].isnull()]['_donationid'].count()

12673

c. Cleanup missing values in donor_state column

In [1]:
for i in df['donor_state'].unique():
    print(i)

NameError: name 'df' is not defined

There are other missing values whose value could not be inferred from other information or columns. Whether to fill them in will depend on the future analysis and machine learning algorithm. 

#### Step 4: Save the optimized dataframe

In [62]:
df.to_csv('project_donation_optimized.csv', index=False)

In [63]:
df.columns

Index(['_donationid', '_projectid', '_donor_acctid', 'donor_city',
       'donor_state', 'is_teacher_acct', 'donation_timestamp',
       'donation_to_project', 'donation_optional_support', 'donation_total',
       'payment_method', 'is_teacher_referred', 'thank_you_packet_mailed',
       '_teacher_acctid', 'school_ncesid', 'school_latitude',
       'school_longitude', 'school_city', 'school_state', 'school_zip',
       'school_metro', 'school_district', 'school_county', 'teacher_prefix',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'total_donations', 'num_donors', 'eligible_double_your_impact_match',
       'eligible_almost_home_match', 'funding_status', 'date_posted',
       'date_completed', 'date_thank_you_packet_mailed', 'date_expiration'

#### Step 5: Create additional columns to help with analysis (pending)